# 2.1 Explicit Intent

W tym przykładzie przyjrzymy się mechanizmowi intentów, rozpoczniemy od intentów jawnych. Stworzymy aplikację z dwiema aktywnościami i umożliwimy otworzenie drugiej aktywności z poziomu pierwszej (głównej). 

Przedźmy do aktywności głównej i dodajmy przycisk do layoutu.

### Layout i utworzenie drugiej aktywności

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <Button
        android:id="@+id/button"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Button"
        android:layout_margin="16dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        android:onClick="startSecondActivity"/>
</androidx.constraintlayout.widget.ConstraintLayout>
```

Po naciśnięciu tego przycisku możliwe będzie przejście do drugiej aktywności.

Następnie dodajmy drugą aktywność do projektu. W oknie **Project** wybieram z menu kontekstowego **New -> Activity -> Empty Activity**. Zaznaczam **Generate layout file** i odznaczam **Launcher Activity**. Przejdźmy do pliku `AndroidManifest.xml`

Mamy opisane dwie aktywności. Chcemy teraz wskazać rodzica dla **SecondActivity** . Do tagu `<activity />` dodaję opcję `android:parentActivityName=".MainActivity"` oraz `android:label="Second Activity"`.

```xml
<activity
    android:name=".SecondActivity"
    android:parentActivityName=".MainActivity"
    android:label="Second Activity"
    android:exported="false" />
```

Czyli wskazaliśmy ze aktywnością nadrzędną jest **MainActivity**. Zwróćmy również uwagę na kropkę przed nazwą aktywności głównej - pominięta została nazwa pakietu (`pl.edu.uwr.pum.explicitintentjava.MainActivity`). `label` posłuży jako wyświetlana nazwa aktywności.

Teraz utworzymy prosty layout dla drugiej aktywności - przechodzimy do pliku `activity_second.xml` i dodajemy pole `TextView`.

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".SecondActivity">

    <TextView
        android:id="@+id/textView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Second Activity"
        android:textSize="40sp"
        android:textStyle="bold"
        android:textColor="@color/black"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />
</androidx.constraintlayout.widget.ConstraintLayout>
```

### Utworzenie `Intent` i przejście do drugiej aktywności

Dodajmy teraz obsługę przycisku w naszej głównej aktywności, przyjdźmy do pliku `MainActivity.java`. Po wciśnięciu przycisku chcemy przejść do drugiej aktywności.

```java
public void startSecondActivity(View view) {
        
}
```

W pierwszym kroku musimy utworzyć **intent**.

```java
Intent intent = new Intent(this, SecondActivity.class);
```

Konstruktor klasy `Intent` przyjmuje dwa argumenty. Pierwszym jest **context**, ponieważ aktywność jest swoim własnym kontekstem używamy słowa kluczowego `this`. Drugim argumentem jest klasa aktywności którą chcemy włączyć - tutaj będzie to `SecondActivity.class`.

Aby włączyć aktywność wykonujemy metodę `startActivity`, jako argumentb podajemy poprzednio utworzony `Intent`.

```java
startActivity(intent);
```

Na tym etapie możemy zbudować nasz projekt i przetestować funkcjonalność przycisku. Po przejściu do drugiej aktywności zwróćmy uwagę na przycisk powrotu znajdujący się na belce po lewej stronie.

<img src="images/explicitIntent1.png" width="200"/>

Nawigację do aktywności nadrzędnej mamy dodaną automatyvcznie dzięki dodaniu do manifestu linii `android:parentActivityName=".MainActivity"`

### Przekazanie danych do drugiej aktywności

Kolejnym krokiem będzie przekazanie danych i ich odbiór w drugiej aktywności - tutaj przekażemy tekst z pola `EditText`. Do layoutu głównej aktywności dodajmy odpowiednie pole.

```xml
    <EditText
        android:id="@+id/editTextMain"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:layout_margin="16dp"
        android:hint="message"
        android:textStyle="bold"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toStartOf="@+id/button"
        app:layout_constraintStart_toStartOf="parent" />
```

Pierwszą rzeczą jakiej będziemy potrzebować jest to **klucz** naszej wiadomości. Przejdźmy do klasy głównej aktyności `MainActivity.java` i dodajmy odpowiednie pole. Standardowo klucze przechowuje się w finalnych polach statycznych klasy wysyłającej.

```java
public static final String EXTRA_MESSAGE = "pl.edu.uwr.pum.explicitintentjava.MESSAGE";
```

Standardowo nazwę stałej przechowującej klucz piszemy wielkimi literami, sam `String` zawierający klucz składa się z pakietu w której znajduje się aktywność wysyłająca i nazwy.

Dalej utwórzmy pole reprezentujące `EditText` i połączmy z elementem layoutu o odpowiednim `id`.

```java
private EditText editText;

@Override
protected void onCreate(Bundle savedInstanceState) {
    super.onCreate(savedInstanceState);
    setContentView(R.layout.activity_main);
        
    editText = findViewById(R.id.editTextMain);
}
```

Ostatnim krokiem będzie spakowanie danych i przesłanie ich do drugiej aktywności. Przechodzimy do metody `startSecondActivity` (czyli metody `onClick` naszego przycisku) i uzupełniamy implementację.

W pierwszym kroku wyciągamy tekst z pola `EditText`

```java
String message = editText.getText().toString();
```

Następnie dodajemy do intentu nasze dane - czyli `String` wyciągnięty z `EditText`. W tym celu na naszym obiekcie `Intent` wywołujemy metodę `putExtra`, która dodaje dane do obiektu `Intent` i przekazuje je do aktywności odbierającej.

```java
intent.putExtra(EXTRA_MESSAGE, message);
```

Metoda przyjmuje dwa argumenty. Pierwszym jest nasz unikalny klucz, który mamy zapisany w stałej `EXTRA_MESSAGE`, drugim jest `String` który chcemy przekazać.

```java
public void startSecondActivity(View view) {
    String message = editText.getText().toString();
    Intent intent = new Intent(this, SecondActivity.class);
    intent.putExtra(EXTRA_MESSAGE, message);
    startActivity(intent);
}
```

### Odebranie danych z poziomu drugiej aktywności

Kolejnym krokiem będzie odebranie danych z poziomu drugiej aktywności - odebrany `String` przekażemy do pola `TextView`. Przechodzimy do pliku `SecondActivity.java` i w metodzie `onCreate` tworzymy `Intent` i wywołujemy metodę `getIntent` - dzięki tej metodzie przesłany `Intent` z aktywności głównej zostanie odebrany.

```java
Intent intent = getIntent();
```

Następnie odbieramy przesłane dane za pomocą metody `getStringExtra` wywołanej na odebranej instancji klasy `Intent`.

```java
String message = intent.getStringExtra(MainActivity.EXTRA_MESSAGE);
```

Metoda jako argument przyjmuje unikalny klucz, który mamy zapisany w stałej statycznej klasy `MainActivity`. W ostatnim kroku chcemy odebrany `String` przekazać do pola `TextView`.

```java
package pl.edu.uwr.pum.explicitintentjava;

import androidx.appcompat.app.AppCompatActivity;

import android.content.Intent;
import android.os.Bundle;
import android.widget.TextView;

public class SecondActivity extends AppCompatActivity {

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_second);

        Intent intent = getIntent();
        String message = intent.getStringExtra(MainActivity.EXTRA_MESSAGE);
        TextView textView = findViewById(R.id.textView);
        textView.setText(message);
    }
}
```

Na tym etapie możemy przetestować funkcjonalność aplikacji.